In [1]:
using PyPlot

In [2]:
push!(LOAD_PATH, ".");
using quantum
names(quantum)

16-element Array{Symbol,1}:
 :P_Orthogonal  
 :P_Unitary     
 :apply_ising!  
 :apply_kick!   
 :apply_unitary!
 :base_state    
 :projector     
 :quantum       
 :random_state  
 :sigma_x       
 :sigma_y       
 :sigma_z       
 :sigmas        
 :staircase     
 :testbit       
 :unfolding     

In [3]:
function concat_ancilla(psi::Array{Complex{Float64},2}, n::Int64)
    psi_anc = zeros(Complex{Float64}, 2^(n+1))
    psi_anc[1:length(psi)] = psi
    return psi_anc
end

concat_ancilla (generic function with 1 method)

In [4]:
function apply_control_not!(psi, control_qubit, target_qubit)
    psi_i_temp = copy(psi)
    mov = max(control_qubit,target_qubit)
    for i = 0:length(psi)-1
        if testbit(i,control_qubit) & !testbit(i,target_qubit)
            psi[i+1] = psi_i_temp[i+1+2^mov]
        elseif testbit(i,control_qubit) & testbit(i,target_qubit)
            psi[i+1] = psi_i_temp[i+1-2^mov]
        end
    end
end

apply_control_not! (generic function with 1 method)

In [49]:
function encoder(psi, k)
    psi_encode = concat_ancilla(psi, 3^k-1)
    
    apply_control_not!(psi_encode, 3*i, 3*i+1)
    
    
    return psi_encode
end

encoder (generic function with 1 method)

In [64]:
function error_generator(Err, psi, p)
    n = Int(log2(length(psi)))
    psi_err = copy(psi)
    for i=0:n-1
        if rand()< p
            apply_unitary!(psi_err, Err, n-1-i)
        end
    end
    return psi_err
end

error_generator (generic function with 1 method)

In [154]:
function partial_trace_pure_bipartite_mat(state, dim, system)
    dimtotal = length(state)[1]
    dimcomp = Int(dimtotal/dim)
    psi = reshape(state,(dimcomp,dim))'
    if system==1
        psi=conj(psi)
        return psi*psi'
    elseif system==2
        return psi'*psi
    end
end

partial_trace_pure_bipartite_mat (generic function with 1 method)

In [ ]:
function error_correction_mesurement!(psi, k)
    psi_anc = concat_ancilla(psi, 2*k)

    apply_transversal_control_not!(psi_anc, 3^(k-1), 0, 4)
    apply_transversal_control_not!(psi_anc, 3^(k-1), 2, 4)
    apply_transversal_control_not!(psi_anc, 3^(k-1), 0, 3)
    apply_transversal_control_not!(psi_anc, 3^(k-1), 1, 3)
    
    ancilla = partial_trace_pure_bipartite_mat(psi_anc, 2^(k+1), 1)
    
    for i in 1:4
        if abs(ancilla[i,i] - 1.0) < 1e-6
            if i == 2
                apply_unitary!(psi, sigma_x, 1)
            elseif i == 3
                apply_unitary!(psi, sigma_x, 2)
            elseif i == 4
                apply_unitary!(psi, sigma_x, 0)
            end
        end
    end
    
    return psi
end

In [170]:
psi = random_state(2)
@time psi = encoder(psi,1)

  0.000011 seconds (9 allocations: 1.094 KB)


8-element Array{Complex{Float64},1}:
 0.389378-0.356615im
      0.0+0.0im     
      0.0+0.0im     
      0.0+0.0im     
      0.0+0.0im     
      0.0+0.0im     
      0.0+0.0im     
 0.393267+0.752696im

In [176]:
@time error_generator(sigma_x, psi, 0.1)

  0.000010 seconds (25 allocations: 1.406 KB)


8-element Array{Complex{Float64},1}:
      0.0+0.0im     
 0.389378-0.356615im
      0.0+0.0im     
      0.0+0.0im     
      0.0+0.0im     
      0.0+0.0im     
 0.393267+0.752696im
      0.0+0.0im     